In [1]:
import os
from func import *

In [25]:
def read_cmd(fp):
    with open(fp, "r") as f:
        temp = f.read()

    return temp

def process_raw_cmd(content):
    n_split = content.split("\n")

    cmds = []

    for i in n_split:
        if i.strip() == "":
            continue
        cmds.append(i.split(" "))

    return cmds

def read_cube(num):
    content = read_file(f"{num}.pcn")
    return read_raw(content)

def opt_and(cube1, cube2, num_var):
    """
    AND(Fn, Fm) = NOT( OR( NOT(Fn), NOT(Fm) ) )
    """
    not_c1 = urp_complement(cube1, num_var)
    not_c2 = urp_complement(cube2, num_var)
    or_them = opt_or(not_c1, not_c2)
    rtn = urp_complement(or_them, num_var)

    return rtn

def process_cmd(list_of_cmd, output_filename):

    var_dict = {}
    # local_num_var = None

    func_ref = {
        "r": lambda x: run_r(*x),
        "!": lambda x: run_complement(*x),
        "+": lambda x: run_or(*x),
        "&": lambda x: run_and(*x),
        "p": lambda x: run_p(*x),
        "q": lambda x: 1
    }

    def run_r(cube_num):
        cube, _num_var = read_cube(int(cube_num))
        var_dict[int(cube_num)] = cube, _num_var

    def run_complement(desc_func_id, input_func_id):
        desc_func_id = int(desc_func_id)
        input_func_id = int(input_func_id)
        num_var = var_dict[input_func_id][1]


        input_func = var_dict[input_func_id][0]
        result_func = urp_complement(input_func, num_var)

        var_dict[desc_func_id] = (result_func, num_var)


    def run_or(desc_func_id, input_func_id_1, input_func_id_2):
        desc_func_id = int(desc_func_id)
        input_func_id_1 = int(input_func_id_1)
        input_func_id_2 = int(input_func_id_2)
        num_var = var_dict[input_func_id_1][1]

        input_func_1 = var_dict[input_func_id_1][0]
        input_func_2 = var_dict[input_func_id_2][0]
        result_func = opt_or(input_func_1, input_func_2)

        var_dict[desc_func_id] = (result_func, num_var)

    def run_and(desc_func_id, input_func_id_1, input_func_id_2):
        desc_func_id = int(desc_func_id)
        input_func_id_1 = int(input_func_id_1)
        input_func_id_2 = int(input_func_id_2)
        num_var = var_dict[input_func_id_1][1]

        input_func_1 = var_dict[input_func_id_1][0]
        input_func_2 = var_dict[input_func_id_2][0]
        result_func = opt_and(input_func_1, input_func_2, num_var)

        var_dict[desc_func_id] = (result_func, num_var)

    def run_p(tar_func_id):
        tar_func_id = int(tar_func_id)
        result, num_var = var_dict[tar_func_id]

        result.insert(0, str(len(result)))
        result.insert(0, str(num_var))
        write_result(result, output_filename)

    for i in list_of_cmd:
        func_ref[i[0]](i[1:])

In [26]:
txts = glob.glob("cmd*.txt")
txts.sort()

In [27]:
for i in range(len(txts)):
    raw = read_cmd(f"cmd{i+1}.txt")
    cmds = process_raw_cmd(raw)
    process_cmd(cmds, f"calpart{i+1}answer.cubes")

In [15]:
process_raw_cmd(read_cmd("cmd5.txt"))

[['r', '3'],
 ['r', '4'],
 ['r', '5'],
 ['!', '6', '3'],
 ['!', '7', '4'],
 ['!', '8', '5'],
 ['&', '1', '3', '4'],
 ['&', '2', '6', '4'],
 ['&', '9', '3', '7'],
 ['&', '10', '6', '7'],
 ['&', '11', '1', '5'],
 ['&', '12', '10', '5'],
 ['&', '13', '2', '8'],
 ['&', '14', '9', '8'],
 ['+', '15', '11', '12'],
 ['+', '16', '13', '14'],
 ['+', '0', '15', '16'],
 ['p', '0'],
 ['q']]

In [16]:
process_cmd(process_raw_cmd(read_cmd("cmd5.txt")), "temp.txt")